In [1]:
from transformers import pipeline
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [2]:
SYNONYMS = {
    'Здравоохранение/Медицина': ['лечение', 'медицинские услуги', 'здравотдел', 'здравпункт', 'мед. обслуживание', 'мин. здрав'],
    'Социальное обслуживание и защита': ['соцобслуживание', 'социальная защита', 'благотворительность', "льготы и пособие", 'социальная поддержка', 'социальная помощь'],
    'Дороги': ['автодороги', 'шоссе', 'трассы', 'проезжая часть', 'дорожная сеть'],
    'ЖКХ': ['коммунальные услуги', 'жилищное хозяйство', 'коммуналка', 'ЖКУ', 'жилищно-коммунальные работы'],
    'Благоустройство': ['озеленение', 'ландшафтный дизайн', 'уличная обстановка', 'уборка территории', 'улучшение территории'],
    'Мусор/Свалки/ТКО': ['отходы', 'утилизация мусора', 'захоронение отходов', 'переработка мусора', 'уборка отходов'],
    'Общественный транспорт': ['пассажирские перевозки', 'общественный транспорт', 'городской транспорт', 'муниципальный транспорт', 'автобусы и троллейбусы', 'оставновки и транспорт'],
    'Коронавирус': ['COVID-19', 'пандемия', 'вирусная инфекция', 'корона', 'эпидемия коронавируса'],
    'Образование': ['педагогика', 'учебный процесс', 'школьное обучение', 'образовательные услуги', 'учебные заведения'],
    'Безопасность': ['охрана порядка', 'профилактика преступлений', 'защита', 'безопасное пространство', 'недопущение нарушений'],
    'Связь и телевидение': ['телекоммуникации', 'масс-медиа', 'информационные сервисы', 'радиовещание', 'информационная индустрия'],
    'Мобилизация': ['призыв', 'военный набор', 'резервисты', 'подготовка к обороне', 'мобилизационная подготовка', 'СВО', 'военная операция'],
    'Физическая культура и спорт': ['спортивная деятельность', 'физкультура', 'спортивные мероприятия', 'физическое воспитание', 'спорт'],
    'Строительство и архитектура': ['застройка', 'архитектурное планирование', 'стройиндустрия', 'градостроительство', 'строительные работы'],
    'Газ и топливо': ['газоснабжение', 'топливная индустрия', 'нефтегаз', 'энергоресурсы', 'газовая промышленность'],
    'Спецпроекты': ['особые программы', 'специальные инициативы', 'эксклюзивные проекты', 'инновационные проекты', 'спецработы'],
    'Культура': ['искусство', 'духовное наследие', 'культурное развитие', 'творчество', 'культурная жизнь'],
    'Электроснабжение': ['энергоснабжение', 'электроэнергия', 'подача тока', 'электросеть', 'энергетика'],
    'Экономика и бизнес': ['экономическая деятельность', 'бизнес-сектор', 'финансы и торговля', 'предпринимательство', 'торгово-экономический комплекс'],
    'Экология': ['охрана окружающей среды', 'природопользование', 'экосистема', 'природозащита', 'экологическая безопасность'],
    'Роспотребнадзор': ['защита прав потребителей', 'санитарный надзор', 'надзор за услугами', 'государственный контроль', 'контроль качества'],
    'Памятники и объекты культурного наследия': ['исторические объекты', 'культурные ценности', 'заповедники', 'архитектурное наследие', 'музей под открытым небом'],
    'Государственная собственность': ['госимущество', 'государственные активы', 'федеральная собственность', 'муниципальное имущество', 'державные резервы'],
    'Торговля': ['коммерция', 'продажи', 'рыночная деятельность', 'внешняя и внутренняя торговля', 'торговый бизнес'],
    'МФЦ "Мои документы"': ['центры обслуживания', 'госуслуги', 'административные сервисы', 'общественные сервисы', 'документационное обслуживание'],
    # 'Погребение и похоронное дело' - для этого термина трудно подобрать синонимы, которые точно передавали бы суть деятельности без контекстной потери.
    'Погребение и похоронное дело': ['ритуальные услуги', 'похоронная служба', 'захоронение', 'ритуальная служба', 'похороны']
}

In [3]:
GROUPS = ['Здравоохранение/Медицина', 'Социальное обслуживание и защита',
          'Дороги', 'ЖКХ', 'Благоустройство', 'Мусор/Свалки/ТКО',
          'Общественный транспорт', 'Коронавирус', 'Образование',
          'Безопасность', 'Культура', 'Связь и телевидение', 'Спецпроекты',
          'Строительство и архитектура', 'Электроснабжение', 'Физическая культура и спорт',
          'Экология', 'Роспотребнадзор', 'МФЦ "Мои документы"', 'Газ и топливо', 'Экономика и бизнес',
          'Государственная собственность', 'Торговля', 'Мобилизация', 'Памятники и объекты культурного наследия',
          'Погребение и похоронное дело']

In [4]:
#2222
#3333
#4444

In [5]:
class MakeZeroShot:
    def __init__(self, hugging_name='cointegrated/rubert-base-cased-nli-threeway'):
        self.tokenizer = AutoTokenizer.from_pretrained(hugging_name)
        
        self.model = AutoModelForSequenceClassification.from_pretrained(hugging_name)
        
        if torch.cuda.is_available():
            self.model = self.model.cuda()
        
        self.text_df = None

    def predict_zero_shot(self, text, label_texts, label='entailment', normalize=True):
        label_texts
        tokens = self.tokenizer([text] * len(label_texts), label_texts, truncation=True, return_tensors='pt', padding=True)
        with torch.inference_mode():
            result = torch.softmax(self.model(**tokens.to(self.model.device)).logits, -1)
        proba = result[:, self.model.config.label2id[label]].cpu().numpy()
        if normalize:
            proba /= sum(proba)
        return proba

    def predict(self, data, presaved=False, dataset_type='train'):
        
        if presaved:
            try:
                path = f"presaved/{dataset_type}_fewshot.csv"
                self.texts_df = pd.read_csv(f"presaved/{dataset_type}_fewshot.csv")
                return pd.concat([data, self.texts_df], axis=1)
            except:
                print(f"No dataset were found! path = {path}")
        
        expanded_labels = ["{} ({})".format(label, ', '.join(SYNONYMS[label])) for label in GROUPS]
        
        texts = data["Текст инцидента"].to_list()

        table_labels = [f"fewshot_{i}" for i, label in enumerate(GROUPS)]

        zero_shot = []
        for text in tqdm(texts):
            zero_shot.append(dict(zip(table_labels, self.predict_zero_shot(text, expanded_labels,
                                                                           label='entailment', normalize=True))))
            
        
        self.texts_df = pd.DataFrame(zero_shot)
        self.texts_df.to_csv(f"presaved/{dataset_type}_fewshot.csv", index=False)
        
        return pd.concat([data, self.texts_df], axis=1)

In [16]:
data = pd.read_csv("final_full.csv")

In [17]:
data.head(2)

,Исполнитель,Группа тем,Текст инцидента,Тема
0,Город Пермь,Погребение и похоронное дело,Погребения - это серьезная проблема в нашей ст...,Погребение и похоронное дело
1,Город Пермь,Погребение и похоронное дело,"Я также считаю, что мы должны делать все возмо...",Погребение и похоронное дело


In [24]:
model.model.device

device(type='cuda', index=0)

In [19]:
new_data = model.predict(data, presaved=True)

In [24]:
new_data.to_csv("train_fewshot.csv", index=False)

In [29]:
new_data.head()

,Исполнитель,Группа тем,Текст инцидента,Тема,fewshot_0,fewshot_1,fewshot_2,fewshot_3,fewshot_4,fewshot_5,...,fewshot_16,fewshot_17,fewshot_18,fewshot_19,fewshot_20,fewshot_21,fewshot_22,fewshot_23,fewshot_24,fewshot_25
0,Город Пермь,Погребение и похоронное дело,Погребения - это серьезная проблема в нашей ст...,Погребение и похоронное дело,0.027873,0.054709,0.181616,0.001006,0.037910,0.013609,...,0.016065,0.003768,0.002787,0.026472,0.004908,0.016191,0.014407,0.009004,0.049282,0.131090
1,Город Пермь,Погребение и похоронное дело,"Я также считаю, что мы должны делать все возмо...",Погребение и похоронное дело,0.084562,0.155073,0.095777,0.001764,0.024469,0.012200,...,0.028868,0.012073,0.004161,0.021829,0.001811,0.015271,0.006325,0.002771,0.051986,0.159060
2,Город Пермь,Погребение и похоронное дело,"Я думаю, что многие люди уже готовы переходить...",Погребение и похоронное дело,0.037693,0.073982,0.149505,0.001130,0.006182,0.017094,...,0.010845,0.003968,0.008324,0.052558,0.005146,0.007187,0.013559,0.004151,0.019356,0.181261
3,Город Пермь,Погребение и похоронное дело,"Я считаю, что строительство новых крематориев ...",Погребение и похоронное дело,0.038174,0.083098,0.171054,0.003283,0.007312,0.017344,...,0.011326,0.006626,0.006078,0.050131,0.031556,0.011899,0.060701,0.005136,0.071414,0.093479
4,Город Пермь,Погребение и похоронное дело,"Я думаю, что мы должны обсуждать этот вопрос о...",Погребение и похоронное дело,0.049650,0.070501,0.142575,0.016416,0.007014,0.020630,...,0.019297,0.003400,0.010802,0.060938,0.027623,0.011117,0.049114,0.011791,0.017505,0.031063


In [33]:
test = pd.read_csv("test.csv", sep=";")

In [35]:
test_data = model.predict(test)

100%|██████████| 9743/9743 [03:22<00:00, 48.23it/s]


In [37]:
test_data = test_data.drop(columns=["id", "Текст инцидента"])

In [39]:
test_data.to_csv("test_fewshot.csv", index=False)

In [23]:
test.to_csv("train")

,id,Текст инцидента
0,0,'Может создать петицию по нашей проблеме)..<br...
1,1,"'Очень надеялась, что недоразумение с Декабрис..."
2,2,'Хоть бы отремонтировали поликлинику на Гайве....
3,3,"'Добрый день! Получаю выплаты с 3 до 7 лет, по..."
4,4,'Добрый день! Подскажите пожалуйста куда обращ...
...,...,...
9738,9738,"'Вдоль Октябрьского 7, до арки, тоже нет освещ..."
9739,9739,'Добрый день !Как сейчас будет происходить вып...
9740,9740,'У нас в посёлке кто-то снял металлические вер...
9741,9741,"'Соликамский р-н, +79523287940. Здравствуйте, ..."


In [14]:
pd.read_csv("presaved/train_fewshot.csv")

,fewshot_0,fewshot_1,fewshot_2,fewshot_3,fewshot_4,fewshot_5,fewshot_6,fewshot_7,fewshot_8,fewshot_9,...,fewshot_16,fewshot_17,fewshot_18,fewshot_19,fewshot_20,fewshot_21,fewshot_22,fewshot_23,fewshot_24,fewshot_25
0,0.027873,0.054709,0.181616,0.001006,0.037910,0.013609,0.084962,0.011668,0.012994,0.116724,...,0.016065,0.003768,0.002787,0.026472,0.004908,0.016191,0.014407,0.009004,0.049282,0.131090
1,0.084562,0.155073,0.095777,0.001764,0.024469,0.012200,0.051186,0.003983,0.013441,0.118982,...,0.028868,0.012073,0.004161,0.021829,0.001811,0.015271,0.006325,0.002771,0.051986,0.159060
2,0.037693,0.073982,0.149505,0.001130,0.006182,0.017094,0.104533,0.007553,0.011407,0.017430,...,0.010845,0.003968,0.008324,0.052558,0.005146,0.007187,0.013559,0.004151,0.019356,0.181261
3,0.038174,0.083098,0.171054,0.003283,0.007312,0.017344,0.064733,0.004413,0.025423,0.051425,...,0.011326,0.006626,0.006078,0.050131,0.031556,0.011899,0.060701,0.005136,0.071414,0.093479
4,0.049650,0.070501,0.142575,0.016416,0.007014,0.020630,0.092648,0.007217,0.025198,0.013005,...,0.019297,0.003400,0.010802,0.060938,0.027623,0.011117,0.049114,0.011791,0.017505,0.031063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24282,0.198019,0.256087,0.076548,0.005927,0.007029,0.005920,0.064194,0.010325,0.009514,0.036870,...,0.012003,0.006627,0.017991,0.005576,0.010555,0.010169,0.014481,0.017121,0.017956,0.007221
24283,0.018982,0.019760,0.079087,0.063861,0.001114,0.224239,0.221832,0.008862,0.004743,0.003997,...,0.003834,0.011699,0.003567,0.010483,0.043464,0.011265,0.057001,0.007541,0.023589,0.016975
24284,0.350882,0.185398,0.053749,0.001299,0.002417,0.003917,0.100051,0.024988,0.012573,0.017864,...,0.006924,0.025657,0.009792,0.012675,0.003082,0.010667,0.007600,0.008683,0.017869,0.009347
24285,0.063323,0.094843,0.132016,0.002472,0.005781,0.017993,0.155446,0.011684,0.074366,0.039513,...,0.011146,0.113917,0.013257,0.019462,0.007839,0.030107,0.014063,0.003395,0.033821,0.002108


In [6]:
model = MakeZeroShot()

In [10]:
new_data = model.predict(test, dataset_type='test')

100%|██████████| 9743/9743 [03:19<00:00, 48.75it/s]


In [9]:
test

,id,Текст инцидента
0,0,'Может создать петицию по нашей проблеме)..<br...
1,1,"'Очень надеялась, что недоразумение с Декабрис..."
2,2,'Хоть бы отремонтировали поликлинику на Гайве....
3,3,"'Добрый день! Получаю выплаты с 3 до 7 лет, по..."
4,4,'Добрый день! Подскажите пожалуйста куда обращ...
...,...,...
9738,9738,"'Вдоль Октябрьского 7, до арки, тоже нет освещ..."
9739,9739,'Добрый день !Как сейчас будет происходить вып...
9740,9740,'У нас в посёлке кто-то снял металлические вер...
9741,9741,"'Соликамский р-н, +79523287940. Здравствуйте, ..."
